In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random

class JointEstimator(nn.Module):
    """Joint model for rapport estimator and social reasoner for social user simulator"""
    def __init__(self, input_size, hidden_size, output_size, leaky_slope, window):
        super(JointEstimator, self).__init__()
        self.alpha = nn.Parameter(torch.FloatTensor(window, 1).fill_(0.1))
        self.beta = nn.Parameter(torch.FloatTensor(window, 1).fill_(0.1))
        self.map1 = nn.Linear(input_size, hidden_size)
        self.activation1 = nn.LeakyReLU(leaky_slope)
        self.map2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, U, A, R):
        u_input = torch.matmul(U, self.alpha[None, :, :]).squeeze()
        a_input = torch.matmul(A, self.beta[None, :, :]).squeeze()
        full_input = torch.cat([u_input, a_input, R], 1)
        output = self.map2(self.activation1(self.map1(full_input)))
        rapp = output[:, 0]
        cs_prob = F.sigmoid(output[:, 1:])
        return rapp, cs_prob

In [7]:
seed = 0
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

input_size = 16 # 6-dimensional user CS, 8-dimensional agent CS, 2-dimensional previous rapport values
hidden_size = 8
output_size = 7 # 6-dimensional user CS (at next step), 1-dimensional rapport value
window = 2 # How many previous user and agent CSs and rapport values we take into account for prediction
leaky_slope = 0.1 # Hyperparameter for Leaky ReLU

je = JointEstimator(input_size, hidden_size, output_size, leaky_slope, window)

In [3]:
loss_fn_rapp = torch.nn.MSELoss()
loss_fn_cs = torch.nn.BCELoss()
optimizer = optim.SGD(je.parameters(), lr=0.05)

In [8]:
# Get data from the pickle file.
import pickle
import os
import json

parent_path = os.path.abspath('../')
data_path = parent_path + '/data/davos/'
train_data_fname = 'train_data_full.pkl'

def add_axis(mat):
    return mat[np.newaxis, :, :]

data = pickle.load(open(data_path + train_data_fname, 'rb'))

# Input (un-processed) to the joint model.
U_full = {}
A_full = {}
R_full = {}

# True values.
u_true = {}
r_true = {}

# Total number of data points.
n_tot = {}

input_variables = ['user_cs_inp', 'agent_cs_inp', 'rapp_inp']
output_variables = ['user_cs_outp', 'rapp_outp']

print(data)

for k, val in data.items():
    all_keys = list(val.keys())
    u_true[k] = val[all_keys[0]]
    r_true[k] = val[all_keys[1]]
    n_tot[k] = u_true[k].shape[0]
    U_full[k] = np.concatenate(( add_axis(val[all_keys[2]]), add_axis(val[all_keys[3]]) ))
    A_full[k] = np.concatenate(( add_axis(val[all_keys[4]]), add_axis(val[all_keys[5]]) ))
    R_full[k] = np.concatenate(( add_axis(val[all_keys[6]]), add_axis(val[all_keys[7]]) ))

{0: {'user_cs_outp': array([[1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]]), 'rapp_outp': array([[5.20253748],
       [5.20253748],
       [4.14325103],
       [4.03152403],
       [4.31290244],
       [5.02793849],
       [5.11160072],
       [5.15505025],
       [5.15922198],
       [5.13320833],
       [4.91675419],
       [4.79274955],
       [4.6650879 ],
       [4.28863879],
       [4.55138323],
       [4.9674992 ],
       [5.60573577],
       [5.42249656],
       [5.18804335],
       [3.99370402],
       [3.8888779 ],
       [3.90855566],
       [3.9609097 ],
       [4.08762204],
       [4.08411909],
       [3.97944352],
       [3.72539443],
       [3.08641975],
       [3.70953086],
       [4.35802469],
       [4.208     ],
       [3.623     ],
       [3.103     ],
       [4.84275309],
       [4.84275309],
       [4.72541798]

In [10]:
import matplotlib.pyplot as plt
clust = 'all'
# x = np.sum(u_true[clust], axis=0)
x = np.average(data[0]['agent_cs_inp_t-0'], axis=0)
print(x)
x = np.average(data[1]['agent_cs_inp_t-0'], axis=0)
print(x)
x = np.average(data['all']['agent_cs_inp_t-0'], axis=0)
print(x)
# plt.bar(['SD', 'QESD', 'PR', 'HE', 'VSN', 'NONE'], x)

x = np.sum(u_true[clust], axis=0)
print(x)

[0.053125   0.084375   0.31041667 0.01875    0.09375    0.
 0.04791667 0.65625   ]
[0.0587156  0.08440367 0.29357798 0.00917431 0.13027523 0.
 0.05321101 0.62018349]
[0.0551495  0.08438538 0.30431894 0.01528239 0.10697674 0.
 0.04983389 0.64318937]
[356.  31.  75.  91.   8. 983.]


Cluster-1: [ 81.  14.  19.  25.   2. 240.] (Just 13 interactions)

Cluster-0: [ 84.   2.   7.  11.   0. 221.] (Total 18 interactions)

In [6]:
clust = 1
def convert_to_tensor(np_array):
    d1, d2, d3 = np_array.shape
    return torch.Tensor(np_array).view(d2, d3, d1)

def re_sample(X, ns):
    return resample(X, replace=True, n_samples=ns)

In [7]:
print(R_full[0].squeeze().transpose().shape)
print(U_full[0].shape)

(322, 2)
(2, 322, 6)


In [12]:
from sklearn.utils import resample

clusters = [0]

for c in clusters:
    u_none = np.where(u_true[c][:, -1] == 1)[0]
    u_not_none = np.where(u_true[c][:, -1] == 0)[0]
    ns = u_none.shape[0]
#     u_true[c] = np.concatenate((re_sample(u_true[c][u_not_none], ns), u_true[c][u_none]))
#     r_true[c] = np.concatenate((re_sample(r_true[c][u_not_none], ns), r_true[c][u_none]))
    R_temp = R_full[0].squeeze().transpose()
    print(u_none)
    print(u_not_none.shape)
    R_f = torch.Tensor(np.concatenate((re_sample(R_temp[u_not_none, :], ns), R_temp[u_none, :])))
    U_full[c] = np.concatenate((re_sample(U_full[c][:, u_not_none, :], ns), U_full[c][:, u_none, :]))
    A_full[c] = np.concatenate((re_sample(A_full[c][:, u_not_none, :], ns), A_full[c][:, u_none, :]))
    
    u_tr_f = torch.Tensor(u_true[clust])
    r_tr_f = torch.Tensor(r_true[clust])
    U_f = convert_to_tensor(U_full[clust])
    A_f = convert_to_tensor(A_full[clust])

[221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238
 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256
 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274
 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292
 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310
 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328
 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346
 347 348 349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364
 365 366 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382
 383 384 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400
 401 402 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418
 419 420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436
 437 438 439 440 441]
(221,)


IndexError: index 322 is out of bounds for axis 0 with size 322

In [ ]:
U_f = convert_to_tensor(U_full[clust])
A_f = convert_to_tensor(A_full[clust])
R_f = torch.Tensor(R_full[clust]).squeeze().transpose(0, 1)
u_tr_f = torch.Tensor(u_true[clust])
r_tr_f = torch.Tensor(r_true[clust])
print(R_f)
print(U_f)

In [ ]:
frac_valid = 0.1
frac_test = 0.1
np.random.seed(0)

def subtract(list1, list2):
    """Subtract list2 from list1"""
    return [i for i in list1 if i not in list2]

n_valid = int(frac_valid * n_tot[clust])
n_test = int(frac_test * n_tot[clust])
n_train = n_tot[clust] - (n_valid + n_test)

all_indices = range(n_tot[clust])
valid_indices = np.random.choice(all_indices, size=n_valid, replace=False).tolist()
rem_indices = subtract(all_indices, valid_indices)
test_indices = np.random.choice(rem_indices, size=n_test, replace=False).tolist()
train_indices = subtract(rem_indices, test_indices)

assert len(valid_indices) == n_valid
assert len(test_indices) == n_test
assert len(train_indices) == n_train

In [ ]:
U = {}
A = {}
R = {}
u_tr = {}
r_tr = {}

data_types = ['train', 'valid', 'test']
indices = [train_indices, valid_indices, test_indices]

for i, dt in enumerate(data_types):
    U[dt] = U_f[torch.LongTensor(indices[i]), :]
    A[dt] = A_f[torch.LongTensor(indices[i]), :]
    R[dt] = R_f[torch.LongTensor(indices[i]), :]
    u_tr[dt] = u_tr_f[torch.LongTensor(indices[i]), :]
    r_tr[dt] = r_tr_f[torch.LongTensor(indices[i]), :]

In [ ]:
# Training loop
input_size = 16
hidden_size = 8
output_size = 7
leaky_slope = 0.1
window = 2
lr = 1e-3

je = JointEstimator(input_size, hidden_size, output_size, leaky_slope, window)

loss_fn_rapp = torch.nn.MSELoss()
loss_fn_cs = torch.nn.BCELoss()
optimizer = optim.Adam(je.parameters(), lr=lr)

for i in range(10000):
    optimizer.zero_grad()
    rapp_pred, cs_pred = je(U[data_types[0]], A[data_types[0]], R[data_types[0]])
    loss_rapp = loss_fn_rapp(rapp_pred, r_tr[data_types[0]].squeeze())
    loss_cs = loss_fn_cs(cs_pred, u_tr[data_types[0]])
    tot_loss = loss_rapp + loss_cs
    tot_loss.backward()
    optimizer.step()
    if i%1000 == 0:
        print(loss_rapp)

In [ ]:
je.eval()

rapp_pred, cs_pred = je(U[data_types[1]], A[data_types[1]], R[data_types[1]])
cs_pred_npy = cs_pred.data.cpu().numpy()
print(cs_pred.topk(5)[1])
print(u_tr[data_types[1]])
loss_rapp = loss_fn_rapp(rapp_pred, r_tr[data_types[1]].squeeze())
loss_cs = loss_fn_cs(cs_pred, u_tr[data_types[1]])
print(loss_rapp)
print(loss_cs)

In [ ]:
rapp_pred, cs_pred = je(U[data_types[2]], A[data_types[2]], R[data_types[2]])
loss_rapp = loss_fn_rapp(rapp_pred, r_tr[data_types[2]].squeeze())
loss_cs = loss_fn_cs(cs_pred, u_tr[data_types[2]])
print(loss_rapp)
print(loss_cs)

In [13]:
import torch
U = torch.randn(322, 6, 2)
zeros = torch.zeros(322, 1)
ones = torch.ones(322, 1)
U_z = U[:, :, 0]
U_o = U[:, :, 1]
U_z_new = torch.cat([U_z, zeros], 1)[:, :, None]
U_o_new = torch.cat([U_o, ones], 1)[:, :, None]
U = torch.cat([U_z_new, U_o_new], 2)
print(U)

tensor([[[-1.7143, -0.3772],
         [ 0.6754, -0.1641],
         [ 1.0356, -0.2542],
         ...,
         [-1.4192,  1.3948],
         [ 1.0090,  1.2068],
         [ 0.0000,  1.0000]],

        [[ 0.4309, -1.1224],
         [ 2.4389,  0.5245],
         [ 0.7458,  2.2433],
         ...,
         [-1.5145,  0.4393],
         [ 1.0415, -0.0892],
         [ 0.0000,  1.0000]],

        [[-0.6727, -1.5005],
         [ 1.7077,  0.9242],
         [-1.8766,  1.0701],
         ...,
         [ 0.8934,  1.7451],
         [-1.4201, -0.7697],
         [ 0.0000,  1.0000]],

        ...,

        [[-1.3901, -1.2300],
         [-1.6119, -0.4785],
         [ 0.0141,  0.6448],
         ...,
         [ 1.5442,  2.1209],
         [-0.4549,  0.8179],
         [ 0.0000,  1.0000]],

        [[-0.1958, -1.6416],
         [-1.0136,  0.4124],
         [-1.4785, -1.1382],
         ...,
         [-0.0728,  2.4634],
         [-1.2678,  0.2790],
         [ 0.0000,  1.0000]],

        [[-1.3525, -0.6478],
       